In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
import os

number_classes = 7 
rows=48
cols=48
batch_size = 16

train_data_direction = './dataset/train'
test_data_direction = './dataset/test'

#Using Data Augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=30,
                                  shear_range=0.3,
                                  zoom_range=0.3,
                                  width_shift_range=0.4,
                                  height_shift_range=0.4,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)


train_generator = train_datagen.flow_from_directory(
        train_data_direction,
        color_mode = 'grayscale',
        target_size = (rows , cols),
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True)

test_generator = test_datagen.flow_from_directory(
        test_data_direction,
        color_mode = 'grayscale',
        target_size = (rows , cols),
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True)



Using TensorFlow backend.


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


28709

## Create Keras Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import ELU
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense

In [14]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal', input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal', input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Conv2D(64, (3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Conv2D(128, (3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Conv2D(256, (3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(64, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


#Softmax Activation in Output
model.add(Dense(number_classes, kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [29]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./checkpoint/emotion_model.h5",
                            monitor="val_loss",
                            mode="min",
                            save_best_only=True,
                            verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                         min_delta = 0,
                         patience = 10,
                         verbose=1,
                         restore_best_weights = True)


reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=3, verbose=1, min_delta=0.0001)

#Create callback list
callbacks = [earlystop, checkpoint, reduce_lr]

model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(lr=0.001),
             metrics=['accuracy'])


total_train_samples = train_generator.samples
total_test_samples = test_generator.samples
epochs = 40


history = model.fit(
            train_generator,
            steps_per_epoch=total_train_samples // batch_size,
            epochs = epochs,
            callbacks = callbacks,
            validation_data = test_generator,
            validation_steps = total_test_samples // batch_size)

Epoch 1/40
1793/1794 [============================>.] - ETA: 0s - loss: 1.2020 - accuracy: 0.5579
Epoch 00001: val_loss improved from inf to 0.97827, saving model to ./checkpoint\emotion_model.h5
1794/1794 [==============================] - 21s 12ms/step - loss: 1.2020 - accuracy: 0.5579 - val_loss: 0.9783 - val_accuracy: 0.6309
Epoch 2/40
1790/1794 [============================>.] - ETA: 0s - loss: 1.2060 - accuracy: 0.5560
Epoch 00002: val_loss improved from 0.97827 to 0.97087, saving model to ./checkpoint\emotion_model.h5
1794/1794 [==============================] - 20s 11ms/step - loss: 1.2059 - accuracy: 0.5560 - val_loss: 0.9709 - val_accuracy: 0.6369
Epoch 3/40
1790/1794 [============================>.] - ETA: 0s - loss: 1.2135 - accuracy: 0.5541
Epoch 00003: val_loss did not improve from 0.97087
1794/1794 [==============================] - 20s 11ms/step - loss: 1.2132 - accuracy: 0.5542 - val_loss: 0.9818 - val_accuracy: 0.6334
Epoch 4/40
1791/1794 [============================

Epoch 27/40
1792/1794 [============================>.] - ETA: 0s - loss: 1.1442 - accuracy: 0.5788
Epoch 00027: val_loss did not improve from 0.94501
1794/1794 [==============================] - 20s 11ms/step - loss: 1.1440 - accuracy: 0.5790 - val_loss: 0.9479 - val_accuracy: 0.6475
Epoch 28/40
1789/1794 [============================>.] - ETA: 0s - loss: 1.1477 - accuracy: 0.5815
Epoch 00028: val_loss did not improve from 0.94501
1794/1794 [==============================] - 20s 11ms/step - loss: 1.1477 - accuracy: 0.5814 - val_loss: 0.9465 - val_accuracy: 0.6484
Epoch 29/40
1794/1794 [==============================] - ETA: 0s - loss: 1.1439 - accuracy: 0.5813
Epoch 00029: val_loss did not improve from 0.94501

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.6000001778593287e-06.
1794/1794 [==============================] - 20s 11ms/step - loss: 1.1439 - accuracy: 0.5813 - val_loss: 0.9451 - val_accuracy: 0.6487
Epoch 30/40
1789/1794 [============================>.] - ETA: 0

# OpenCV Integration

In [30]:
from tensorflow.keras.models import load_model

classifier = load_model('./checkpoint/emotion_model.h5')

In [38]:
test_generator = test_datagen.flow_from_directory(
        test_data_direction,
        color_mode = 'grayscale',
        target_size = (rows , cols),
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True)

class_labels = test_generator.class_indices
class_labels = {v : k for k,v in class_labels.items()}
classes = list(class_labels.values())
print(class_labels)

Found 7178 images belonging to 7 classes.
{0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}


In [40]:
import cv2
import numpy as np
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')

def face_detector(img):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for(x,y,w,h) in faces:
        x = x-50
        w = w+50
        y = y-50
        h = h+50
        cv2.rectangle(img, (x,y), (x+w,y+h) , (255,0,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        
    try:
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation = cv2.INTER_AREA)
    except:
        return (x,w,y,h), np.zeros((48,48), np.uint8), img
    return (x,w,y,h), roi_gray, img

cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    if np.sum([face]) != 0.0:
        roi = face.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        preds = classifier.predict(roi)[0]
        label = class_labels[preds.argmax()]
        label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
        cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
    else:
        cv2.putText(image, "No Face found", (20,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
        
    cv2.imshow('Emotion Detector', image)
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()
    
    
    
    
    
    
